In [85]:
#Se importan las librerías utilizadas:

import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import tqdm 
import time
from natsort import natsorted
from natsort import index_natsorted, order_by_index
import cx_Oracle
from datetime import datetime
from datetime import timedelta
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)

In [2]:
cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle\instantclient_21_6")

In [3]:
#Se establecen las fechas de corte y un ajuste manual de pólizas del exterior PREGUNTAR
Ajuste_manual_polizas_exterior = 3017239080.8

#Se define el patrimonio como medida de control para la retenciÓn neta
#Se actualizarÁ por lo menos una vez al aÑo
patrimonio = 94616197580

#Fecha para la extracci�n de bases
fecha_0 = pd.to_datetime("1/01/2023", format='%d/%m/%Y')

fecha_ini_0 =  pd.to_datetime("01/01/2011", format='%d/%m/%Y')


In [10]:
#Se recuperan las bases de datos descargadas
# Desde AQUI
caracteristicas<-caracteristicasBU
caracteristicas_IAXIS <- caracteristicas_IAXISBU
facultativo<-facultativoBU
polizas<-polizasBU
polizas_IAXIS<-polizas_IAXISBU
proporcionales<-proporcionalesBU
asegurado<-aseguradoBU
trm<-trmBU



In [4]:
#Consulta Proporcionales  Oracle:

connect_string = """
    (DESCRIPTION=
    (ADDRESS_LIST= 
        (ADDRESS=(PROTOCOL=TCP)(HOST=172.18.23.194)(PORT=1521)))
    (CONNECT_DATA=(SID=SEGCONF2)))"""

con = cx_Oracle.connect( "csalazar", "Csalazar", connect_string)

cursor=con.cursor()
cursor.arraysize = 2000

cursor.execute("""SELECT O.CONTRATO, O.REASEGURADOR, O.PARTICIPACION, 
		A.SIGLA NOMBRE, A.NIT
    FROM CONFIANZA.S04003 O
	LEFT JOIN (SELECT SIGLA, CODIGO, NIT FROM CONFIANZA.S03500
		WHERE SIGLA IN (SELECT MAX(SIGLA) FROM CONFIANZA.S03500
		GROUP BY CODIGO)) A
		ON O.REASEGURADOR = A.CODIGO""",)

#Se convierte a dataframe:
col=[]

for i in range(0,len(cursor.description)):

    col.append(cursor.description[i][0])

proporcionalesBU=pd.DataFrame(data=cursor,index=None,columns=col)

cursor.close()

con.close()

In [5]:
#Consulta Proporcionales  Oracle:

connect_string = """
    (DESCRIPTION=
    (ADDRESS_LIST= 
        (ADDRESS=(PROTOCOL=TCP)(HOST=172.18.23.194)(PORT=1521)))
    (CONNECT_DATA=(SID=SEGCONF2)))"""

con = cx_Oracle.connect( "csalazar", "Csalazar", connect_string)

cursor=con.cursor()
cursor.arraysize = 2000

cursor.execute("""SELECT O.CONTRATO, O.REASEGURADOR, O.PARTICIPACION, 
		A.SIGLA NOMBRE, A.NIT
    FROM CONFIANZA.S04003 O
	LEFT JOIN (SELECT SIGLA, CODIGO, NIT FROM CONFIANZA.S03500
		WHERE SIGLA IN (SELECT MAX(SIGLA) FROM CONFIANZA.S03500
		GROUP BY CODIGO)) A
		ON O.REASEGURADOR = A.CODIGO""",)

#Se convierte a dataframe:
col=[]

for i in range(0,len(cursor.description)):

    col.append(cursor.description[i][0])

proporcionalesBU=pd.DataFrame(data=cursor,index=None,columns=col)

cursor.close()

con.close()

In [6]:
#Consulta Proporcionales  Oracle:

connect_string = """
    (DESCRIPTION=
    (ADDRESS_LIST= 
        (ADDRESS=(PROTOCOL=TCP)(HOST=172.18.23.194)(PORT=1521)))
    (CONNECT_DATA=(SID=SEGCONF2)))"""

con = cx_Oracle.connect( "csalazar", "Csalazar", connect_string)

cursor=con.cursor()
cursor.arraysize = 2000

cursor.execute(""" SELECT 
    F.SUCUR, B.CER_NUM_POLIZA POLIZA, 
    (SELECT H.PERVIN_PERSONA FROM BDG.DIM_PERSONA_VINCULACION H WHERE H.PERVIN_ID=B.CER_FK_ASEGURADO) NIT_ASE, 
    (SELECT MAX(I.PER_PERSONA_NOMBRE) FROM BDG.DIM_PERSONA_VINCULACION H, BDG.DIM_PERSONA I
          WHERE I.PER_CODIGO=H.PERVIN_CODIGO
          AND H.PERVIN_ID=B.CER_FK_ASEGURADO
          AND I.ESTADO='ACTIVO'
          AND I.PER_PERSONA=H.PERVIN_PERSONA) ASEGURADO
  FROM
    BDG.FACT_CERTIFICADOS B,
  --BDG.FACT_CESIONES C,
    BDG.DIM_TIPO_CERTIFICADO D,
    BDG.DIM_TECNICA E,
  (
    SELECT
      GEO_ID,
      CASE WHEN GEO_AGENCIA IS NULL THEN GEO_SUCURSAL ELSE GEO_AGENCIA END AS SUCUR
    FROM
      BDG.DIM_GEOGRAFICA
    WHERE
      ESTADO = 'ACTIVO'
  ) F
  WHERE
    B.CER_FK_TECNICA = E.TEC_ID
 AND B.CER_FK_GEOGRAFICA = F.GEO_ID
  AND E.TEC_RAMO in ('03','07','08','09','11','14','15','071') """,)

#Se convierte a dataframe:
col=[]

for i in range(0,len(cursor.description)):

    col.append(cursor.description[i][0])

aseguradoBU=pd.DataFrame(data=cursor,index=None,columns=col)

cursor.close()

con.close()

In [28]:
fecha_ini_0

Timestamp('2011-01-01 00:00:00')

In [93]:
#Consulta Proporcionales  Oracle:

connect_string = """
    (DESCRIPTION=
    (ADDRESS_LIST= 
        (ADDRESS=(PROTOCOL=TCP)(HOST=172.18.23.194)(PORT=1521)))
    (CONNECT_DATA=(SID=SEGCONF2)))"""

con = cx_Oracle.connect( "csalazar", "Csalazar", connect_string)

cursor=con.cursor()
cursor.arraysize = 2000


#Fecha para la extracci�n de bases
F_FIN1=pd.to_datetime("01/01/2023", format='%d/%m/%Y')
F_INI1=pd.to_datetime("01/01/2011", format='%d/%m/%Y')

cursor.execute(""" SELECT A.CODRAM, A.SUCUR, A.ORDEN, B.POLIZA, A.CERTIF, A.INCISO, B.TIPCER, A.CODCLA,
			B.FECEXP, A.FECINI, A.FECFIN, A.VASEG, A.PRIMA, B.VPRIMA PRIMACOP, B.TIPCOA, B.COASEG, 
			B.ANTCERTIF, A.MONVAL, A.DEDUCP DEDUCIBLE, B.CONT1, B.CONT2, B.CONT3, B.CONT4, B.CONT5,
			B.VA1, B.VA2, B.VA3, B.VA4, B.VA5, B.VAF, C.FECREN
		FROM CONFIANZA.S03033 A
		LEFT JOIN (SELECT B.CODRAM, B.SUCUR, B.ORDEN, B.CERTIF, B.POLIZA, B.TIPCER, B.FECEXP,
			B.TIPCOA, B.COASEG, B.ANTCERTIF, B.VPRIMA, B.CONT1, B.CONT2, B.CONT3, B.CONT4, B.CONT5,
			B.VA1, B.VA2, B.VA3, B.VA4, B.VA5, B.VAF FROM CONFIANZA.S04021 B) B
			ON (A.CODRAM = B.CODRAM
			AND A.SUCUR = B.SUCUR
			AND A.ORDEN = B.ORDEN
			AND A.CERTIF = B.CERTIF)
		LEFT JOIN (SELECT C.SUCUR, C.ORDEN, C.CERTIF, C.FECREN FROM CONFIANZA.S03020 C) C
			ON (A.SUCUR = C.SUCUR
			AND A.ORDEN = C.ORDEN
			AND A.CERTIF = C.CERTIF)
		WHERE
		A.CODRAM IN ('03','08', '14')
		AND A.CODCLA IN ('07010040', '07010150', '07010110', '03010090') """,)

#Se convierte a dataframe:
col=[]

for i in range(0,len(cursor.description)):

    col.append(cursor.description[i][0])

polizas=pd.DataFrame(data=cursor,index=None,columns=col)


#Filtramos solo para las pólizas del periodo de interes
polizas['FECHA_CONTABLE'] = polizas[['FECEXP', 'FECINI']].max(axis=1)
polizas = polizas[(polizas.FECHA_CONTABLE <= F_FIN1)&(polizas.FECHA_CONTABLE >= F_INI1)]

cursor.close()

con.close()

polizas



,CODRAM,SUCUR,ORDEN,POLIZA,CERTIF,INCISO,TIPCER,CODCLA,FECEXP,FECINI,FECFIN,VASEG,PRIMA,PRIMACOP,TIPCOA,COASEG,ANTCERTIF,MONVAL,DEDUCIBLE,CONT1,CONT2,CONT3,CONT4,CONT5,VA1,VA2,VA3,VA4,VA5,VAF,FECREN,FECHA_CONTABLE
0,08,103,1,PT000504,PT000506,1,N,07010040,2018-12-28,2018-12-21,2019-12-20,2.105191e+06,4055.0,92928410.0,3.0,10.0,None,2,0.0,RTI2018,CUT2018,None,None,None,0.0,0.0,0.0,0.0,0.0,100.0,2018-12-21,2018-12-28
1,08,103,1,PT000504,PT000506,2,N,07010040,2018-12-28,2018-12-21,2019-12-20,1.198178e+07,23080.0,92928410.0,3.0,10.0,None,2,0.0,RTI2018,CUT2018,None,None,None,0.0,0.0,0.0,0.0,0.0,100.0,2018-12-21,2018-12-28
2,08,103,1,PT000504,PT000506,3,N,07010040,2018-12-28,2018-12-21,2019-12-20,1.440000e+05,277.0,92928410.0,3.0,10.0,None,2,0.0,RTI2018,CUT2018,None,None,None,0.0,0.0,0.0,0.0,0.0,100.0,2018-12-21,2018-12-28
3,03,124,1,CM000795,CM001662,0,M,03010090,2018-12-04,2018-11-27,2018-11-30,3.166871e+08,0.0,0.0,1.0,0.0,CM001650,1,3.0,RETTR18,CTRCM18,None,None,None,0.0,100.0,0.0,0.0,0.0,0.0,2018-11-27,2018-12-04
4,03,102,1,CM000694,CM001200,0,N,03010090,2019-01-14,2018-12-18,2019-09-23,1.072264e+10,0.0,11794901.0,1.0,0.0,None,1,3.0,RETTR18,CTRCM18,None,None,None,0.0,100.0,0.0,0.0,0.0,0.0,2018-12-18,2019-01-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103318,08,106,1,PZ000528,PZ000690,3,M,07010040,2022-12-29,2022-11-30,2022-12-15,3.266642e+10,270146.0,2071121.0,3.0,9.5,PZ000684,1,0.0,RTI2021,CUT2021,None,None,None,0.0,100.0,0.0,0.0,0.0,0.0,2021-12-01,2022-12-29
103319,08,106,1,PZ000528,PZ000690,4,M,07010040,2022-12-29,2022-11-30,2022-12-15,6.910902e+08,337683.0,2071121.0,3.0,9.5,PZ000684,1,0.0,RTI2021,CUT2021,None,None,None,0.0,100.0,0.0,0.0,0.0,0.0,2021-12-01,2022-12-29
103320,08,106,1,PZ000528,PZ000690,5,M,07010040,2022-12-29,2022-11-30,2022-12-15,1.188194e+10,337683.0,2071121.0,3.0,9.5,PZ000684,1,0.0,RTI2021,CUT2021,None,None,None,0.0,100.0,0.0,0.0,0.0,0.0,2021-12-01,2022-12-29
103321,08,106,1,PZ000528,PZ000690,6,M,07010040,2022-12-29,2022-11-30,2022-12-15,1.225500e+07,337683.0,2071121.0,3.0,9.5,PZ000684,1,0.0,RTI2021,CUT2021,None,None,None,0.0,100.0,0.0,0.0,0.0,0.0,2021-12-01,2022-12-29


In [66]:
#Consulta Proporcionales  Oracle:

connect_string = """
    (DESCRIPTION=
    (ADDRESS_LIST= 
        (ADDRESS=(PROTOCOL=TCP)(HOST=172.18.23.194)(PORT=1521)))
    (CONNECT_DATA=(SID=SEGCONF2)))"""

con = cx_Oracle.connect( "csalazar", "Csalazar", connect_string)

cursor=con.cursor()
cursor.arraysize = 2000

#Fecha para la extracci�n de bases
F_FIN1=pd.to_datetime("2023-01-01", format='%Y-%m-%d')
F_INI1=pd.to_datetime("2011-01-01", format='%Y-%m-%d')

cursor.execute(""" SELECT DISTINCT
    E.TEC_RAMO CODRAM,	
    F.SUCUR SUCUR,
    B.CER_NUM_ORDEN ORDEN,
    CAST(B.CER_NUM_POLIZA  AS VARCHAR2(15)) POLIZA,
    B.CER_NUM_CERTIFICADO CERTIF,
    1 AS INCISO,
    D.TCE_TIPO_CERT TIPCER,
    '07010040' AS CODCLA,
    B.CER_FECHA_EXPEDICION FECEXP,
    B.CER_FECHA_DESDE FECINI,
    B.CER_FECHA_HASTA FECFIN,
    0 AS VASEG,
    B.CER_PRIMA_DIRECTA PRIMA,
    B.CER_PRIMA_DIRECTA PRIMACOP,
    B.CER_TIPO_COASEGURO TIPCOA,
    B.CER_PORC_COASEGURO COASEG,
    CASE WHEN D.TCE_TIPO_CERT='A' THEN B.CER_NUM_CERTIFICADO ELSE NULL END AS ANTCERTIF,
    CASE WHEN B.CER_FK_MONEDA=156 THEN 2 ELSE 1 END AS MONVAL,
    B.CER_TRM_EXPED,
    (SELECT ROUND(MAX(CASE WHEN G.COB_VALOR_ASEGURADO=0 THEN 0 ELSE 100*G.COB_VALOR_DEDU/G.COB_VALOR_ASEGURADO END),1)
 FROM BDG.FACT_COBERTURAS G
     WHERE G.COB_FK_TECNICA IN (SELECT DISTINCT TEC_ID FROM BDG.DIM_TECNICA WHERE TEC_RAMO='08')
    AND G.COB_NUM_POLIZA=B.CER_NUM_POLIZA) DEDUCIBLE,
    'RTI2022' AS CONT1,
    'CUT2022' AS CONT2,
    0 AS CONT3,
    0 AS CONT4,
    0 AS CONT5,
    C.CES_PORC_VA1 VA1,
    C.CES_PORC_VA2 VA2,
    C.CES_PORC_VA3 VA3,
    C.CES_PORC_VA4 VA4,
    C.CES_PORC_VA5 VA5,
    C.CES_PORC_VAFA VAF,
    (CASE WHEN D.TCE_TIPO_CERT='R' THEN (CASE WHEN B.CER_FECHA_EXPEDICION>=B.CER_FECHA_DESDE THEN B.CER_FECHA_EXPEDICION ELSE B.CER_FECHA_EXPEDICION END)
    ELSE NULL END) AS FECREN
FROM
BDG.FACT_CERTIFICADOS B LEFT JOIN 
BDG.FACT_CESIONES C
    ON B.CER_FK_GEOGRAFICA = C.CES_FK_GEOGRAFICA
    AND B.CER_NUM_CERTIFICADO = C.CES_NUM_CERTIFICADO
    AND B.CER_NUM_ORDEN = C.CES_NUM_ORDEN
    AND B.CER_NUM_POLIZA = C.CES_NUM_POLIZA
    AND B.CER_FK_TIEMPO = C.CES_FK_TIEMPO,
BDG.DIM_TIPO_CERTIFICADO D,
BDG.DIM_TECNICA E,
(
  SELECT
  GEO_ID,
  CASE WHEN GEO_AGENCIA IS NULL THEN GEO_SUCURSAL ELSE GEO_AGENCIA END AS SUCUR
  FROM
  BDG.DIM_GEOGRAFICA
  WHERE
  ESTADO = 'ACTIVO'
) F
WHERE
    B.CER_FK_TIPOCERTIFICADO = D.TCE_ID
    AND B.CER_FK_TECNICA = E.TEC_ID
    AND B.CER_FK_GEOGRAFICA = F.GEO_ID
    AND E.TEC_RAMO = '08'
    AND SUBSTR(B.CER_NUM_POLIZA,1,3)='807' """,)

#Se convierte a dataframe:
col=[]

for i in range(0,len(cursor.description)):

    col.append(cursor.description[i][0])

polizas_IAXIS=pd.DataFrame(data=cursor,index=None,columns=col)


#Filtramos solo para las pólizas del periodo de interes
polizas_IAXIS['FECHA_CONTABLE'] = polizas_IAXIS[['FECEXP', 'FECINI']].max(axis=1)
polizas_IAXIS = polizas_IAXIS[(polizas_IAXIS.FECHA_CONTABLE <= F_FIN1)&(polizas_IAXIS.FECHA_CONTABLE >= F_INI1)]

cursor.close()

con.close()

In [67]:
#Hacemos la consulta para la base asegurable

connect_string = """
    (DESCRIPTION=
    (ADDRESS_LIST= 
        (ADDRESS=(PROTOCOL=TCP)(HOST=172.18.23.194)(PORT=1521)))
    (CONNECT_DATA=(SID=SEGCONF2)))"""

con = cx_Oracle.connect( "csalazar", "Csalazar", connect_string)

cursor=con.cursor()
cursor.arraysize = 2000

#Fecha para la extracci�n de bases
F_FIN1=pd.to_datetime("2023-01-01", format='%Y-%m-%d')
F_INI1=pd.to_datetime("2011-01-01", format='%Y-%m-%d')

cursor.execute(""" SELECT DISTINCT
E.TEC_RAMO CODRAM,
F.SUCUR SUCUR,
B.CER_NUM_ORDEN ORDEN,
B.CER_NUM_POLIZA POLIZA,
B.CER_NUM_CERTIFICADO CERTIF,
1 AS INCISO,
D.TCE_TIPO_CERT TIPCER,
'07010040' AS CODCLA,
B.CER_FECHA_EXPEDICION FECEXP,
B.CER_FECHA_DESDE FECINI,
B.CER_FECHA_HASTA FECFIN,
B.CER_VALOR_ASEGURADO ASEGURABLE,
B.CER_PRIMA_DIRECTA PRIMA,
B.CER_PRIMA_DIRECTA PRIMACOP,
B.CER_TIPO_COASEGURO TIPCOA,
B.CER_PORC_COASEGURO COASEG

FROM
BDG.FACT_CERTIFICADOS B LEFT JOIN
BDG.FACT_CESIONES C
ON B.CER_FK_GEOGRAFICA = C.CES_FK_GEOGRAFICA
AND B.CER_NUM_CERTIFICADO = C.CES_NUM_CERTIFICADO
AND B.CER_NUM_ORDEN = C.CES_NUM_ORDEN
AND B.CER_NUM_POLIZA = C.CES_NUM_POLIZA
AND B.CER_FK_TIEMPO = C.CES_FK_TIEMPO,
BDG.DIM_TIPO_CERTIFICADO D,
BDG.DIM_TECNICA E,
(
  SELECT
  GEO_ID,
  CASE WHEN GEO_AGENCIA IS NULL THEN GEO_SUCURSAL ELSE GEO_AGENCIA END AS SUCUR
  FROM
  BDG.DIM_GEOGRAFICA
  WHERE
  ESTADO = 'ACTIVO'
) F
WHERE
B.CER_FK_TIPOCERTIFICADO = D.TCE_ID
AND B.CER_FK_TECNICA = E.TEC_ID
AND B.CER_FK_GEOGRAFICA = F.GEO_ID
AND E.TEC_RAMO IN ('08','14')
AND SUBSTR(B.CER_NUM_POLIZA,1,3)='807' """,)

#Se convierte a dataframe:
col=[]

for i in range(0,len(cursor.description)):

    col.append(cursor.description[i][0])

BASE_ASEGURABLE=pd.DataFrame(data=cursor,index=None,columns=col)


#Filtramos solo para las pólizas del periodo de interes
BASE_ASEGURABLE['FECHA_CONTABLE'] = BASE_ASEGURABLE[['FECEXP', 'FECINI']].max(axis=1)
BASE_ASEGURABLE = BASE_ASEGURABLE[(BASE_ASEGURABLE.FECHA_CONTABLE <= F_FIN1)&(BASE_ASEGURABLE.FECHA_CONTABLE >= F_INI1)]

cursor.close()

con.close()

In [100]:
BASE_ASEGURABLE['LLAVE'] = BASE_ASEGURABLE.POLIZA.astype('str') + BASE_ASEGURABLE.CERTIF.astype('str') + BASE_ASEGURABLE.ORDEN.astype('str')
polizas_IAXIS['LLAVE'] = polizas_IAXIS.POLIZA.astype('str') + polizas_IAXIS.CERTIF.astype('str') + polizas_IAXIS.ORDEN.astype('str')
#UNIMOS LA BASE CONTABLE DE IAXIS CON LAS POLIZAS DE IAXIS

In [105]:
polizas_IAXIS.merge(BASE_ASEGURABLE[['LLAVE', 'ASEGURABLE']], on = 'LLAVE').fillna(0).dtypes

CODRAM                    object
SUCUR                     object
ORDEN                      int64
POLIZA                    object
CERTIF                    object
INCISO                     int64
TIPCER                    object
CODCLA                    object
FECEXP                    object
FECINI            datetime64[ns]
FECFIN            datetime64[ns]
VASEG                      int64
PRIMA                    float64
PRIMACOP                 float64
TIPCOA                     int64
COASEG                   float64
ANTCERTIF                 object
MONVAL                     int64
CER_TRM_EXPED            float64
DEDUCIBLE                float64
CONT1                     object
CONT2                     object
CONT3                      int64
CONT4                      int64
CONT5                      int64
VA1                      float64
VA2                      float64
VA3                        int64
VA4                        int64
VA5                        int64
VAF       